In [ ]:
# -*- coding: utf-8 -*-
"""
Created on Tue May 14 12:16:11 2019

@author: viryl
"""

from __future__ import print_function
import os
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from datetime import datetime
from ipdb import set_trace





In [ ]:
from torch.utils.data import Dataset as BaseDataset


# 加载输入

class MyDataset(BaseDataset):
    """CamVid Dataset. Read images, apply augmentation and preprocessing transformations.

    Args:
        images_dir (str): path to images folder
        masks_dir (str): path to segmentation masks folder
        class_values (list): values of classes to extract from segmentation mask
        augmentation (albumentations.Compose): data transfromation pipeline
            (e.g. flip, scale, etc.)
        preprocessing (albumentations.Compose): data preprocessing
            (e.g. noralization, shape manipulation, etc.)

    """

    def __init__(
            self,
            images_npy,
            label_npy,

           
    ):


        self.images = images_npy
        self.labels = label_npy
        self.length = images_npy.shape[0]
        # convert str names to class values on masks



    def __getitem__(self, i):
#a= MyDataset(img, label), a[0]
        # read data
        image = self.images[i]

        label =  self.labels[i]

         
        
   
        
        
        
        return torch.from_numpy(image), torch.from_numpy(np.array(label))
        
           



    def __len__(self):
        return self.length

# 返回一个patch 和label, a= MyDataset(img, label), len(a)


  
    # 包含patch image和相应label的元组

In [ ]:
import matplotlib.pyplot as plt

from ipdb import set_trace



def validate(net, data_loader, set_name, classes_name):
    """
    对一批数据进行预测，返回混淆矩阵以及Accuracy
    :param net:
    :param data_loader:
    :param set_name:  eg: 'valid' 'train' 'tesst
    :param classes_name:
    :return:
    """
    net.eval()
    cls_num = len(classes_name)
    conf_mat = np.zeros([cls_num, cls_num])

    for data in data_loader:
        images, labels = data
        if torch.cuda.is_available():
            images, labels = images.cuda(), labels.cuda()
        outputs = net(images)
        outputs.detach_()
        _, predicted = torch.max(outputs.data, 1)

        # 统计混淆矩阵
        for i in range(len(labels)):
            cate_i = labels[i]
            pre_i = predicted[i]
            conf_mat[cate_i, pre_i] += 1.0

    for i in range(cls_num):
        print('class:{:<10}, total num:{:<6}, correct num:{:<5}  Recall: {:.2%} Precision: {:.2%}'.format(
            classes_name[i], np.sum(
                conf_mat[i, :]), conf_mat[i, i], conf_mat[i, i] / (1 + np.sum(conf_mat[i, :])),
            conf_mat[i, i] / (1 + np.sum(conf_mat[:, i]))))

    print('{} set Accuracy:{:.2%}'.format(
        set_name, np.trace(conf_mat) / np.sum(conf_mat)))

    return conf_mat, '{:.2}'.format(np.trace(conf_mat) / np.sum(conf_mat))


# 生成图像
def show_confMat(confusion_mat, classes, set_name, out_dir):

    # 归一化
    confusion_mat_N = confusion_mat.copy()
    for i in range(len(classes)):
        confusion_mat_N[i, :] = confusion_mat[i, :] / confusion_mat[i, :].sum()

    # 获取颜色
    # 更多颜色: http://matplotlib.org/examples/color/colormaps_reference.html
    cmap = plt.cm.get_cmap('Greys')
    plt.imshow(confusion_mat_N, cmap=cmap)
    plt.colorbar()

    # 设置文字
    xlocations = np.array(range(len(classes)))
    plt.xticks(xlocations, list(classes), rotation=60)
    plt.yticks(xlocations, list(classes))
    plt.xlabel('Predict label')
    plt.ylabel('True label')
    plt.title('Confusion_Matrix_' + set_name)

    # 打印数字
    for i in range(confusion_mat_N.shape[0]):
        for j in range(confusion_mat_N.shape[1]):
            plt.text(x=j, y=i, s=int(
                confusion_mat[i, j]), va='center', ha='center', color='red', fontsize=10)
    # 保存
    plt.savefig(os.path.join(out_dir, 'Confusion_Matrix' + set_name + '.png'))
    plt.show()
    


In [ ]:
import torch

class ManifoldLayer(object):
    def __init__(self, ReductionMethod):
        self.ReductionMethod = ReductionMethod
        
        
    def __call__(self, x, *arg, train = False):
        _x = x.cpu().clone().detach().numpy()
        
        _x = _x.reshape(_x.shape[0]*_x.shape[1], _x.shape[2]*_x.shape[3])
        if train:    
            
            self.M = self.ReductionMethod(_x, *arg)
            self.M = np.array(self.M)
            
        
        __x = np.dot(_x, self.M)
        __x  = __x.reshape(x.shape[0], int(__x.shape[0] / x.shape[0]), int(sqrt(__x.shape[1])) , int(sqrt(__x.shape[1])))
        return torch.from_numpy(__x).cuda()
#         x = torch.tensor(x)
#         x = torch.matmul(x.reshape(6000, 576), m1.T)
    
  
     
        


    

In [ ]:
import transplant
matlab = transplant.Matlab()
def myMFA(channel,dim, kw, kb):
    def calMatrix(x,y):
        y = [np.argmax(one_hot)for one_hot in y]
        y = np.array(y).reshape(len(y), 1)
        
       
        return matlab.MFA(x.astype(np.double), y.astype(np.double), channel,dim, kw, kb)
    return calMatrix

In [ ]:
from sklearn.decomposition import PCA

def myPCA(dim):
    def calMatrix(X):
        pca.fit(X)
        M = pca.components_
        return M.T
    pca = PCA(n_components=dim, svd_solver = 'auto')    
    return calMatrix


In [ ]:
import torch.nn as nn
import torch.nn.functional as F
from ipdb import set_trace
from math import sqrt       
     
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 6, 5) 
        self.conv2 = nn.Conv2d(6, 16, 5).double() 
        self.fc1   = nn.Linear(256,128)  
        self.fc2   = nn.Linear(128, 10)
#         self.manifold1 = ManifoldLayer(myMFA(6, 256, 5, 10))
#         self.manifold2 = ManifoldLayer(myMFA(16, 16, 5, 10))   
        self.manifold1 = ManifoldLayer(myPCA(256))
        self.manifold2 = ManifoldLayer(myPCA(16))  

    def forward(self, x, *y, train=False): 
        self.batchsize = x.shape[0]
        x = self.conv1(x)
        
      #  ============================
        x = self.manifold1(x, *y, train=train)
       
       # =================================
       
        x = self.conv2(x)
    #    ===================================
        x = self.manifold2(x, *y, train=train)
        

       # ===================================
        x = x.view(x.size()[0], -1)   #展开成一维的
        
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))        
        return x


net = Net()
print(net)

In [ ]:
from __future__ import print_function
import os
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from datetime import datetime
import sklearn
from sklearn.utils.random import sample_without_replacement
import numpy
import gzip


# Params for MNIST
IMAGE_SIZE = 28
NUM_CHANNELS = 1
PIXEL_DEPTH = 255
NUM_LABELS = 10
# Extract the images
def extract_data(filename, num_images):
    """Extract the images into a 4D tensor [image index, y, x, channels].
    Values are rescaled from [0, 255] down to [-0.5, 0.5].
    """
    print('Extracting', filename)
    with gzip.open(filename) as bytestream:
        bytestream.read(16)
        buf = bytestream.read(IMAGE_SIZE * IMAGE_SIZE * num_images * NUM_CHANNELS)
        data = numpy.frombuffer(buf, dtype=numpy.uint8).astype(numpy.float32)
        data = (data - (PIXEL_DEPTH / 2.0)) / PIXEL_DEPTH
        data = data.reshape(num_images, IMAGE_SIZE, IMAGE_SIZE, NUM_CHANNELS)
        data = numpy.reshape(data, [num_images, -1])
    return data
def extract_labels(filename, num_images):
    """Extract the labels into a vector of int64 label IDs."""
    print('Extracting', filename)
    with gzip.open(filename) as bytestream:
        bytestream.read(8)
        buf = bytestream.read(1 * num_images)
        labels = numpy.frombuffer(buf, dtype=numpy.uint8).astype(numpy.int64)
        num_labels_data = len(labels)
        one_hot_encoding = numpy.zeros((num_labels_data,NUM_LABELS))
        one_hot_encoding[numpy.arange(num_labels_data),labels] = 1
        one_hot_encoding = numpy.reshape(one_hot_encoding, [-1, NUM_LABELS])
    return one_hot_encoding
x_train = extract_data(r'F:\lzy\data\mnist/train-images-idx3-ubyte.gz', 60000)
y_train = extract_labels(r'F:\lzy\data\mnist/train-labels-idx1-ubyte.gz', 60000)
x_test = extract_data(r'F:\lzy\data\mnist/t10k-images-idx3-ubyte.gz', 10000)
y_test = extract_labels(r'F:\lzy\data\mnist/t10k-labels-idx1-ubyte.gz', 10000)






In [ ]:
# 定义超参
EPOCH = 1000
BATCH_SIZE = 100
classes_name = [str(c) for c in range(10)]  # 分类地物数量

In [ ]:
x_train = x_train.reshape(60000, 1, 28, 28)
x_test = x_test.reshape(10000, 1, 28, 28)

In [ ]:
import warnings 
warnings.filterwarnings('ignore')
from tensorboardX import  SummaryWriter

training_dataset = MyDataset(x_train, y_train)
testing_dataset = MyDataset(x_test, y_test)
# Data Loaders
train_loader = torch.utils.data.DataLoader(
    dataset=training_dataset,
    batch_size=BATCH_SIZE,
    shuffle=True
)

test_loader = torch.utils.data.DataLoader(
    dataset=testing_dataset,
    batch_size=BATCH_SIZE,
    shuffle=True
)

# 检查cuda是否可用

# 生成log
now_time = datetime.now()
time_str = datetime.strftime(now_time, '%m-%d_%H-%M-%S')
log_path = os.path.join(os.getcwd(), "log")
log_dir = os.path.join(log_path, time_str)
if not os.path.exists(log_dir):
    os.makedirs(log_dir)
writer = SummaryWriter(log_dir)




# ---------------------搭建网络--------------------------
cnn = Net()  # 创建CNN, 输入全连接层维数

cnn = cnn.double()




# --------------------设置损失函数和优化器----------------------
optimizer = optim.Adam(cnn.parameters(), lr = 0.01)  # lr:(default: 1e-3)优化器
criterion = nn.CrossEntropyLoss()  # 损失函数
scheduler = torch.optim.lr_scheduler.StepLR(
    optimizer, step_size=EPOCH/2, gamma=0.5)  # 设置学习率下降策略

# --------------------训练------------------------------
 # 使用GPU
cnn = cnn.cuda()
for epoch in range(EPOCH):
    loss_sigma = 0.0    # 记录一个epoch的loss之和
    correct = 0.0
    total = 0.0
    scheduler.step()  # 更新学习率

    for batch_idx, data in enumerate(train_loader):
        # 获取图片和标签
        inputs, labels = data
        inputs = inputs.double()
        labels = labels.double()
        
        inputs, labels = inputs.cuda(), labels.cuda()
        optimizer.zero_grad()  # 清空梯度
        cnn = cnn.train()
        label_numpy = labels.cpu().numpy()
        
#         outputs = cnn(inputs,  label_numpy , train=True)
        outputs = cnn(inputs, train=True)
        loss = criterion(outputs, torch.max(labels, 1)[1])
     
        loss.backward()  # 反向传播
        optimizer.step()  # 更新权值
        loss_sigma += loss.item()
        # 统计预测信息
    final_outputs = cnn(torch.from_numpy(x_train).double().cuda(), train=False)
    final_outputs.detach_() 
    _, predicted = torch.max(final_outputs, 1)
    
    total = y_train.shape[0]
   
    correct = ((predicted ==  torch.max(torch.from_numpy(y_train).cuda(), 1)[1]).squeeze().sum()).item()
    

        # 每 BATCH_SIZE 个 iteration 打印一次训练信息，loss为 BATCH_SIZE 个 iteration 的平均   
    loss_avg = loss_sigma / BATCH_SIZE
    
    print("Training: Epoch[{:0>3}/{:0>3}] Iteration[{:0>3}/{:0>3}] Loss: {:.4f} Acc:{:.2%}".format(
        epoch + 1, EPOCH, batch_idx + 1, len(train_loader), loss_sigma, correct / total))
    # 记录训练loss
    writer.add_scalars(
        'Loss_group', {'train_loss': loss_avg}, epoch)
    # 记录learning rate
    writer.add_scalar(
        'learning rate', scheduler.get_lr()[0], epoch)
    # 记录Accuracy
    writer.add_scalars('Accuracy_group', {
                       'train_acc': correct / total}, epoch)
    # 每个epoch，记录梯度，权值
#     for name, layer in cnn.named_parameters():
#         writer.add_histogram(
#             name + '_grad', layer.grad.cpu().data.numpy(), epoch)
#         writer.add_histogram(name + '_data', layer.cpu().data.numpy(), epoch)

    # ------------------------------------ 观察模型在验证集上的表现 ------------------------------------
    if epoch % 1 == 0:
        loss_sigma = 0.0
        cls_num = len(classes_name)
        conf_mat = np.zeros([cls_num, cls_num])  # 混淆矩阵
        cnn.eval()
        for batch_idx, data in enumerate(test_loader):
            images, labels = data
             
            
            images, labels = images.cuda(), labels.cuda()
            images = images.double()
            labels = labels.double()
            cnn.cuda()
            cnn = cnn.train()
            
            outputs = cnn(images)  # forward
           
            outputs.detach_()  # 不求梯度
            
            loss = criterion(outputs, torch.max(labels, 1)[1])  # 计算loss
            loss_sigma += loss.item()

            _, predicted = torch.max(outputs.data, 1)  # 统计
            # labels = labels.data    # Variable --> tensor
            # 统计混淆矩阵
            for j in range(len(labels)):
                
                
                cate_i =  torch.max(labels, 1)[1][j]
                pre_i =  predicted[j]
                conf_mat[int(cate_i), int(pre_i)] += 1.0
        print('{} set Accuracy:{:.2%}'.format(
            'Valid', conf_mat.trace() / conf_mat.sum()))
        # 记录Loss, accuracy
        writer.add_scalars(
            'Loss_group', {'valid_loss': loss_sigma / len(test_loader)}, epoch)
        writer.add_scalars('Accuracy_group', {
                           'valid_acc': conf_mat.trace() / conf_mat.sum()}, epoch)
print('Finished Training')

# ----------------------- 保存模型 并且绘制混淆矩阵图 -------------------------
cnn_save_path = os.path.join(log_dir, 'net_params.pkl')
torch.save(cnn.state_dict(), cnn_save_path)

conf_mat_train, train_acc = F(cnn, train_loader, 'train', classes_name)
conf_mat_valid, valid_acc = validate(cnn, test_loader, 'test', classes_name)

show_confMat(conf_mat_train, classes_name, 'train', log_dir)
show_confMat(conf_mat_valid, classes_name, 'valid', log_dir)

Training: Epoch[001/1000] Iteration[600/600] Loss: 1321.8424 Acc:10.44%
Valid set Accuracy:10.28%
Training: Epoch[002/1000] Iteration[600/600] Loss: 1303.5724 Acc:14.18%
Valid set Accuracy:14.19%
Training: Epoch[003/1000] Iteration[600/600] Loss: 1300.1789 Acc:25.71%
Valid set Accuracy:25.89%
